In [224]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
tf.config.set_visible_devices([], 'GPU')

# scaling 과정 

In [331]:
def LOG(data, feature_list):
    DATA = data.copy()
    for i in feature_list:
        DATA[i] = pd.DataFrame(np.log(DATA[i]))
    return DATA

In [332]:
log_list = ['Tg','age','bmi', 'sys', 'dia','Hba1c', 'FBS']

In [333]:
# ###### FBS autoencoder 

# INCOME
# TOTALC
# PACKYR
# SBP
# DBP
# PULSE
# BMI
# HBA1C
# GLU0
# CREATININE
# AST
# ALT
# TCHL
# HDL
# TRIGLY
# WBC_B
# HB
# PLAT
# INS0
# OBDIG1
# OBDIG2
# IB1_4
# EXER_FINAL

In [334]:
# import plotly.express as px
# fig = px.box(Koges_ansan_clinical_train_data.drop(columns = ['DIST_ID','Tg', 'age', 'bmi','sys','dia','FBS']))
# fig.show()

# 1] model 1 

In [430]:
log_list = ['Tg','age','bmi', 'sys', 'dia','Hba1c', 'FBS']

In [431]:
# koges_ansan 
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
Koges_ansan_clinical_train_data = pd.read_csv(data_path+"/clinical_data/Koges_ansan_1_clinical_train_data_5836_14.csv")
Koges_ansan_clinical_test_data = pd.read_csv(data_path+"/clinical_data/Koges_1_clinical_test_data_1166_14.csv")
print('Koges_ansan_clinical_train_data_shape :', Koges_ansan_clinical_train_data.shape)
print('Koges_ansan_clinical_test_data_shape :', Koges_ansan_clinical_test_data.shape)

Koges_ansan_clinical_train_data_shape : (5836, 15)
Koges_ansan_clinical_test_data_shape : (1166, 15)


In [432]:
Koges_ansan_clinical_train_data.columns

Index(['DIST_ID', 'age', 'sex', 'bmi', 'sys', 'dia', 'htndiag', 'lipdiag',
       'exercise', 'smoke', 'drink', 'Hba1c', 'FBS', 'Tg', 'progress_DM'],
      dtype='object')

In [433]:
# Koges clinical 
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
Koges_clinical_train_data = pd.read_csv(data_path+"/clinical_data/Koges_1_clinical_train_data_4664_14.csv")
Koges_clinical_test_data = pd.read_csv(data_path+"/clinical_data/Koges_1_clinical_test_data_1166_14.csv")
print('Koges_clinical_train_data_shape :', Koges_clinical_train_data.shape)
print('Koges_clinical_test_data_shape :', Koges_clinical_test_data.shape)

Koges_clinical_train_data_shape : (4664, 15)
Koges_clinical_test_data_shape : (1166, 15)


In [434]:
# Koges clinical + snp
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
Koges_clinical_snp_train_data = pd.read_csv(data_path+"/clinical_snp_data/Koges_1_clinical_snp_train_data_4664_46.csv")
Koges_clinical_snp_test_data = pd.read_csv(data_path+"/clinical_snp_data/Koges_1_clinical_snp_test_data_1166_46.csv")
print('Koges_clinical_snp_train_data_shape :', Koges_clinical_snp_train_data.shape)
print('Koges_clinical_snp_test_data_shape :', Koges_clinical_snp_test_data.shape)

Koges_clinical_snp_train_data_shape : (4664, 47)
Koges_clinical_snp_test_data_shape : (1166, 47)


# 1. KoGES ansan clinical 

In [435]:
train_data = Koges_ansan_clinical_train_data.copy()
test_data = Koges_ansan_clinical_test_data.copy()

In [436]:
tr_x, tr_y = train_data.drop(columns = ['progress_DM']), train_data['progress_DM']
ts_x, ts_y = test_data.drop(columns = ['progress_DM']), test_data['progress_DM']

In [437]:
loged_tr_x = LOG(tr_x, log_list)
loged_ts_x = LOG(ts_x, log_list)

In [438]:
minmax_scaler = MinMaxScaler()
minmax_tr_x = minmax_scaler.fit_transform(loged_tr_x.drop(columns = ['DIST_ID']))
minmax_ts_x = minmax_scaler.transform(loged_ts_x.drop(columns = ['DIST_ID']))

In [439]:
columns = loged_tr_x.drop(columns = ['DIST_ID']).columns.tolist()

In [440]:
scaled_train_data = pd.concat([loged_tr_x['DIST_ID'], pd.DataFrame(minmax_tr_x,columns = columns),tr_y], axis=1)
scaled_test_data = pd.concat([loged_ts_x['DIST_ID'], pd.DataFrame(minmax_ts_x,columns = columns),ts_y], axis=1)

In [441]:
print(scaled_train_data.shape)
print(scaled_test_data.shape)

(5836, 15)
(1166, 15)


In [442]:
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
scaled_train_data.to_csv(data_path+"/clinical_data/scaled_log_minmax_Koges_ansan_1_clinical_train_data_5836_14.csv",index=False)

# 2. KoGES clinical 

In [443]:
train_data = Koges_clinical_train_data.copy()
test_data = Koges_clinical_test_data.copy()
print(train_data.shape)
print(test_data.shape)

(4664, 15)
(1166, 15)


In [444]:
tr_x, tr_y = train_data.drop(columns = ['progress_DM']), train_data['progress_DM']
ts_x, ts_y = test_data.drop(columns = ['progress_DM']), test_data['progress_DM']

In [445]:
loged_tr_x = LOG(tr_x, log_list)
loged_ts_x = LOG(ts_x, log_list)

In [446]:
minmax_scaler = MinMaxScaler()
minmax_tr_x = minmax_scaler.fit_transform(loged_tr_x.drop(columns = ['DIST_ID']))
minmax_ts_x = minmax_scaler.transform(loged_ts_x.drop(columns = ['DIST_ID']))

In [447]:
columns = loged_tr_x.drop(columns = ['DIST_ID']).columns.tolist()

In [448]:
scaled_train_data = pd.concat([loged_tr_x['DIST_ID'], pd.DataFrame(minmax_tr_x,columns = columns),tr_y], axis=1)
scaled_test_data = pd.concat([loged_ts_x['DIST_ID'], pd.DataFrame(minmax_ts_x,columns = columns),ts_y], axis=1)

In [449]:
print(scaled_train_data.shape)
print(scaled_test_data.shape)

(4664, 15)
(1166, 15)


In [450]:
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
scaled_train_data.to_csv(data_path+"/clinical_data/scaled_log_minmax_Koges_1_clinical_train_data_4664_14.csv",index=False)
scaled_test_data.to_csv(data_path+"/clinical_data/scaled_log_minmax_Koges_1_clinical_test_data_1166_14.csv",index=False)

# 2. KoGES clinical +snp 

In [451]:
train_data = Koges_clinical_snp_train_data.copy()
test_data = Koges_clinical_snp_test_data.copy()
print(train_data.shape)
print(test_data.shape)

(4664, 47)
(1166, 47)


In [452]:
tr_x, tr_y = train_data.drop(columns = ['progress_DM']), train_data['progress_DM']
ts_x, ts_y = test_data.drop(columns = ['progress_DM']), test_data['progress_DM']

In [453]:
loged_tr_x = LOG(tr_x, log_list)
loged_ts_x = LOG(ts_x, log_list)

In [454]:
minmax_scaler = MinMaxScaler()
minmax_tr_x = minmax_scaler.fit_transform(loged_tr_x.drop(columns = ['DIST_ID']))
minmax_ts_x = minmax_scaler.transform(loged_ts_x.drop(columns = ['DIST_ID']))

In [455]:
columns = loged_tr_x.drop(columns = ['DIST_ID']).columns.tolist()

In [456]:
scaled_train_data = pd.concat([loged_tr_x['DIST_ID'], pd.DataFrame(minmax_tr_x,columns = columns),tr_y], axis=1)
scaled_test_data = pd.concat([loged_ts_x['DIST_ID'], pd.DataFrame(minmax_ts_x,columns = columns),ts_y], axis=1)

In [457]:
print(scaled_train_data.shape)
print(scaled_test_data.shape)

(4664, 47)
(1166, 47)


In [458]:
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
scaled_train_data.to_csv(data_path+"/clinical_snp_data/scaled_log_minmax_Koges_1_clinical_snp_train_data_4664_46.csv",index=False)
scaled_test_data.to_csv(data_path+"/clinical_snp_data/scaled_log_minmax_Koges_1_clinical_snp_test_data_1166_46.csv",index=False)

# 2] model 2

In [459]:
log_list = ['Tg','age','bmi', 'sys', 'dia','Hba1c', 'FBS','WBC', 'ALT','Cr']

In [460]:
# koges_ansan 
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
Koges_ansan_clinical_train_data = pd.read_csv(data_path+"/clinical_data/Koges_ansan_2_clinical_train_data_5835_17.csv")
Koges_ansan_clinical_test_data = pd.read_csv(data_path+"/clinical_data/Koges_2_clinical_test_data_1166_17.csv")
print('Koges_ansan_clinical_train_data_shape :', Koges_ansan_clinical_train_data.shape)
print('Koges_ansan_clinical_test_data_shape :', Koges_ansan_clinical_test_data.shape)

Koges_ansan_clinical_train_data_shape : (5835, 18)
Koges_ansan_clinical_test_data_shape : (1166, 18)


In [461]:
Koges_ansan_clinical_train_data.columns

Index(['DIST_ID', 'age', 'sex', 'bmi', 'sys', 'dia', 'htndiag', 'lipdiag',
       'exercise', 'smoke', 'drink', 'Hba1c', 'FBS', 'Tg', 'WBC', 'ALT', 'Cr',
       'progress_DM'],
      dtype='object')

In [462]:
# Koges clinical 
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
Koges_clinical_train_data = pd.read_csv(data_path+"/clinical_data/Koges_2_clinical_train_data_4663_17.csv")
Koges_clinical_test_data = pd.read_csv(data_path+"/clinical_data/Koges_2_clinical_test_data_1166_17.csv")
print('Koges_clinical_train_data_shape :', Koges_clinical_train_data.shape)
print('Koges_clinical_test_data_shape :', Koges_clinical_test_data.shape)

Koges_clinical_train_data_shape : (4663, 18)
Koges_clinical_test_data_shape : (1166, 18)


In [463]:
# Koges clinical + snp
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
Koges_clinical_snp_train_data = pd.read_csv(data_path+"/clinical_snp_data/Koges_2_clinical_snp_train_data_4663_49.csv")
Koges_clinical_snp_test_data = pd.read_csv(data_path+"/clinical_snp_data/Koges_2_clinical_snp_test_data_1166_49.csv")
print('Koges_clinical_snp_train_data_shape :', Koges_clinical_snp_train_data.shape)
print('Koges_clinical_snp_test_data_shape :', Koges_clinical_snp_test_data.shape)

Koges_clinical_snp_train_data_shape : (4663, 50)
Koges_clinical_snp_test_data_shape : (1166, 50)


# 1. KoGES ansan clinical 

In [464]:
train_data = Koges_ansan_clinical_train_data.copy()
test_data = Koges_ansan_clinical_test_data.copy()

In [465]:
tr_x, tr_y = train_data.drop(columns = ['progress_DM']), train_data['progress_DM']
ts_x, ts_y = test_data.drop(columns = ['progress_DM']), test_data['progress_DM']

In [466]:
loged_tr_x = LOG(tr_x, log_list)
loged_ts_x = LOG(ts_x, log_list)

In [467]:
minmax_scaler = MinMaxScaler()
minmax_tr_x = minmax_scaler.fit_transform(loged_tr_x.drop(columns = ['DIST_ID']))
minmax_ts_x = minmax_scaler.transform(loged_ts_x.drop(columns = ['DIST_ID']))

In [468]:
columns = loged_tr_x.drop(columns = ['DIST_ID']).columns.tolist()

In [469]:
scaled_train_data = pd.concat([loged_tr_x['DIST_ID'], pd.DataFrame(minmax_tr_x,columns = columns),tr_y], axis=1)
scaled_test_data = pd.concat([loged_ts_x['DIST_ID'], pd.DataFrame(minmax_ts_x,columns = columns),ts_y], axis=1)

In [470]:
print(scaled_train_data.shape)
print(scaled_test_data.shape)

(5835, 18)
(1166, 18)


In [471]:
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
scaled_train_data.to_csv(data_path+"/clinical_data/scaled_log_minmax_Koges_ansan_2_clinical_train_data_5835_17.csv",index=False)

# 2. KoGES clinical 

In [472]:
train_data = Koges_clinical_train_data.copy()
test_data = Koges_clinical_test_data.copy()
print(train_data.shape)
print(test_data.shape)

(4663, 18)
(1166, 18)


In [473]:
tr_x, tr_y = train_data.drop(columns = ['progress_DM']), train_data['progress_DM']
ts_x, ts_y = test_data.drop(columns = ['progress_DM']), test_data['progress_DM']

In [474]:
loged_tr_x = LOG(tr_x, log_list)
loged_ts_x = LOG(ts_x, log_list)

In [475]:
minmax_scaler = MinMaxScaler()
minmax_tr_x = minmax_scaler.fit_transform(loged_tr_x.drop(columns = ['DIST_ID']))
minmax_ts_x = minmax_scaler.transform(loged_ts_x.drop(columns = ['DIST_ID']))

In [476]:
columns = loged_tr_x.drop(columns = ['DIST_ID']).columns.tolist()

In [477]:
scaled_train_data = pd.concat([loged_tr_x['DIST_ID'], pd.DataFrame(minmax_tr_x,columns = columns),tr_y], axis=1)
scaled_test_data = pd.concat([loged_ts_x['DIST_ID'], pd.DataFrame(minmax_ts_x,columns = columns),ts_y], axis=1)

In [478]:
print(scaled_train_data.shape)
print(scaled_test_data.shape)

(4663, 18)
(1166, 18)


In [479]:
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
scaled_train_data.to_csv(data_path+"/clinical_data/scaled_log_minmax_Koges_2_clinical_train_data_4663_17.csv",index=False)
scaled_test_data.to_csv(data_path+"/clinical_data/scaled_log_minmax_Koges_2_clinical_test_data_1166_17.csv",index=False)

# 2. KoGES clinical +snp 

In [480]:
train_data = Koges_clinical_snp_train_data.copy()
test_data = Koges_clinical_snp_test_data.copy()
print(train_data.shape)
print(test_data.shape)

(4663, 50)
(1166, 50)


In [481]:
tr_x, tr_y = train_data.drop(columns = ['progress_DM']), train_data['progress_DM']
ts_x, ts_y = test_data.drop(columns = ['progress_DM']), test_data['progress_DM']

In [482]:
loged_tr_x = LOG(tr_x, log_list)
loged_ts_x = LOG(ts_x, log_list)

In [483]:
minmax_scaler = MinMaxScaler()
minmax_tr_x = minmax_scaler.fit_transform(loged_tr_x.drop(columns = ['DIST_ID']))
minmax_ts_x = minmax_scaler.transform(loged_ts_x.drop(columns = ['DIST_ID']))

In [484]:
columns = loged_tr_x.drop(columns = ['DIST_ID']).columns.tolist()

In [485]:
scaled_train_data = pd.concat([loged_tr_x['DIST_ID'], pd.DataFrame(minmax_tr_x,columns = columns),tr_y], axis=1)
scaled_test_data = pd.concat([loged_ts_x['DIST_ID'], pd.DataFrame(minmax_ts_x,columns = columns),ts_y], axis=1)

In [486]:
print(scaled_train_data.shape)
print(scaled_test_data.shape)

(4663, 50)
(1166, 50)


In [487]:
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
scaled_train_data.to_csv(data_path+"/clinical_snp_data/scaled_log_minmax_Koges_2_clinical_snp_train_data_4663_49.csv",index=False)
scaled_test_data.to_csv(data_path+"/clinical_snp_data/scaled_log_minmax_Koges_2_clinical_snp_test_data_1166_49.csv",index=False)

# 3] model 3

In [488]:
# import plotly.express as px
# fig = px.box(Koges_ansan_clinical_train_data.drop(columns = ['DIST_ID','Tg', 'age', 'bmi','sys','dia','FBS']))
# fig.show()

In [489]:
log_list = ['Tg','age','bmi', 'sys', 'dia','Hba1c', 'FBS','WBC', 'ALT','Cr','F_insulin', 'Plt']

In [490]:
# koges_ansan 
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
Koges_ansan_clinical_train_data = pd.read_csv(data_path+"/clinical_data/Koges_ansan_3_clinical_train_data_5112_19.csv")
Koges_ansan_clinical_test_data = pd.read_csv(data_path+"/clinical_data/Koges_3_clinical_test_data_1166_19.csv")
print('Koges_ansan_clinical_train_data_shape :', Koges_ansan_clinical_train_data.shape)
print('Koges_ansan_clinical_test_data_shape :', Koges_ansan_clinical_test_data.shape)

Koges_ansan_clinical_train_data_shape : (5112, 20)
Koges_ansan_clinical_test_data_shape : (1166, 20)


In [491]:
Koges_ansan_clinical_train_data.columns

Index(['DIST_ID', 'age', 'sex', 'bmi', 'sys', 'dia', 'htndiag', 'lipdiag',
       'exercise', 'smoke', 'drink', 'Hba1c', 'FBS', 'Tg', 'WBC', 'ALT', 'Cr',
       'F_insulin', 'Plt', 'progress_DM'],
      dtype='object')

In [492]:
# Koges clinical 
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
Koges_clinical_train_data = pd.read_csv(data_path+"/clinical_data/Koges_3_clinical_train_data_4662_19.csv")
Koges_clinical_test_data = pd.read_csv(data_path+"/clinical_data/Koges_3_clinical_test_data_1166_19.csv")
print('Koges_clinical_train_data_shape :', Koges_clinical_train_data.shape)
print('Koges_clinical_test_data_shape :', Koges_clinical_test_data.shape)

Koges_clinical_train_data_shape : (4662, 20)
Koges_clinical_test_data_shape : (1166, 20)


In [493]:
# Koges clinical + snp
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
Koges_clinical_snp_train_data = pd.read_csv(data_path+"/clinical_snp_data/Koges_3_clinical_snp_train_data_4662_51.csv")
Koges_clinical_snp_test_data = pd.read_csv(data_path+"/clinical_snp_data/Koges_3_clinical_snp_test_data_1166_51.csv")
print('Koges_clinical_snp_train_data_shape :', Koges_clinical_snp_train_data.shape)
print('Koges_clinical_snp_test_data_shape :', Koges_clinical_snp_test_data.shape)

Koges_clinical_snp_train_data_shape : (4662, 52)
Koges_clinical_snp_test_data_shape : (1166, 52)


# 1. KoGES ansan clinical 

In [494]:
train_data = Koges_ansan_clinical_train_data.copy()
test_data = Koges_ansan_clinical_test_data.copy()

In [495]:
tr_x, tr_y = train_data.drop(columns = ['progress_DM']), train_data['progress_DM']
ts_x, ts_y = test_data.drop(columns = ['progress_DM']), test_data['progress_DM']

In [496]:
loged_tr_x = LOG(tr_x, log_list)
loged_ts_x = LOG(ts_x, log_list)

In [497]:
minmax_scaler = MinMaxScaler()
minmax_tr_x = minmax_scaler.fit_transform(loged_tr_x.drop(columns = ['DIST_ID']))
minmax_ts_x = minmax_scaler.transform(loged_ts_x.drop(columns = ['DIST_ID']))

In [498]:
columns = loged_tr_x.drop(columns = ['DIST_ID']).columns.tolist()

In [499]:
scaled_train_data = pd.concat([loged_tr_x['DIST_ID'], pd.DataFrame(minmax_tr_x,columns = columns),tr_y], axis=1)
scaled_test_data = pd.concat([loged_ts_x['DIST_ID'], pd.DataFrame(minmax_ts_x,columns = columns),ts_y], axis=1)

In [500]:
print(scaled_train_data.shape)
print(scaled_test_data.shape)

(5112, 20)
(1166, 20)


In [501]:
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
scaled_train_data.to_csv(data_path+"/clinical_data/scaled_log_minmax_Koges_ansan_3_clinical_train_data_5112_19.csv",index=False)


# 2. KoGES clinical 

In [502]:
train_data = Koges_clinical_train_data.copy()
test_data = Koges_clinical_test_data.copy()
print(train_data.shape)
print(test_data.shape)

(4662, 20)
(1166, 20)


In [503]:
tr_x, tr_y = train_data.drop(columns = ['progress_DM']), train_data['progress_DM']
ts_x, ts_y = test_data.drop(columns = ['progress_DM']), test_data['progress_DM']

In [504]:
loged_tr_x = LOG(tr_x, log_list)
loged_ts_x = LOG(ts_x, log_list)

In [505]:
minmax_scaler = MinMaxScaler()
minmax_tr_x = minmax_scaler.fit_transform(loged_tr_x.drop(columns = ['DIST_ID']))
minmax_ts_x = minmax_scaler.transform(loged_ts_x.drop(columns = ['DIST_ID']))

In [506]:
columns = loged_tr_x.drop(columns = ['DIST_ID']).columns.tolist()

In [507]:
scaled_train_data = pd.concat([loged_tr_x['DIST_ID'], pd.DataFrame(minmax_tr_x,columns = columns),tr_y], axis=1)
scaled_test_data = pd.concat([loged_ts_x['DIST_ID'], pd.DataFrame(minmax_ts_x,columns = columns),ts_y], axis=1)

In [508]:
print(scaled_train_data.shape)
print(scaled_test_data.shape)

(4662, 20)
(1166, 20)


In [509]:
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
scaled_train_data.to_csv(data_path+"/clinical_data/scaled_log_minmax_Koges_3_clinical_train_data_4662_19.csv",index=False)
scaled_test_data.to_csv(data_path+"/clinical_data/scaled_log_minmax_Koges_3_clinical_test_data_1166_19.csv",index=False)

# 2. KoGES clinical +snp 

In [510]:
train_data = Koges_clinical_snp_train_data.copy()
test_data = Koges_clinical_snp_test_data.copy()
print(train_data.shape)
print(test_data.shape)

(4662, 52)
(1166, 52)


In [511]:
tr_x, tr_y = train_data.drop(columns = ['progress_DM']), train_data['progress_DM']
ts_x, ts_y = test_data.drop(columns = ['progress_DM']), test_data['progress_DM']

In [512]:
loged_tr_x = LOG(tr_x, log_list)
loged_ts_x = LOG(ts_x, log_list)

In [513]:
minmax_scaler = MinMaxScaler()
minmax_tr_x = minmax_scaler.fit_transform(loged_tr_x.drop(columns = ['DIST_ID']))
minmax_ts_x = minmax_scaler.transform(loged_ts_x.drop(columns = ['DIST_ID']))

In [514]:
columns = loged_tr_x.drop(columns = ['DIST_ID']).columns.tolist()

In [515]:
scaled_train_data = pd.concat([loged_tr_x['DIST_ID'], pd.DataFrame(minmax_tr_x,columns = columns),tr_y], axis=1)
scaled_test_data = pd.concat([loged_ts_x['DIST_ID'], pd.DataFrame(minmax_ts_x,columns = columns),ts_y], axis=1)

In [516]:
print(scaled_train_data.shape)
print(scaled_test_data.shape)

(4662, 52)
(1166, 52)


In [517]:
data_path = '/nfs-data/Glucose/SW1-2/Ehkang/data/new_11_08'
scaled_train_data.to_csv(data_path+"/clinical_snp_data/scaled_log_minmax_Koges_3_clinical_snp_train_data_4663_51.csv",index=False)
scaled_test_data.to_csv(data_path+"/clinical_snp_data/scaled_log_minmax_Koges_3_clinical_snp_test_data_1166_51.csv",index=False)

In [518]:
scaled_train_data

,DIST_ID,rs340874,rs780094,rs4402960,rs1470579,rs10946398,rs1799884,rs4607517,rs7034200,rs2237895,...,drink,Hba1c,FBS,Tg,WBC,ALT,Cr,F_insulin,Plt,progress_DM
0,NIH20F9182533,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,...,1.0,1.000000,0.960043,0.733664,0.539543,0.547902,0.510023,0.530361,0.776271,1
1,NIH20F9862958,0.0,0.5,0.0,0.0,0.0,1.0,1.0,0.5,0.5,...,0.0,0.939097,0.791325,0.295284,0.426433,0.255193,0.292373,0.641102,0.609645,0
2,NIH20F9404770,0.0,1.0,0.0,0.0,0.5,0.0,0.0,0.5,0.0,...,0.5,0.843955,0.460043,0.108319,0.635115,0.578819,0.578648,0.521243,0.613100,1
3,NIH20F9367586,0.0,0.5,0.5,0.5,1.0,0.0,0.0,0.0,0.0,...,0.0,0.876196,0.781585,0.292775,0.381315,0.182558,0.496312,0.627823,0.653949,0
4,NIH20F9763035,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,...,1.0,0.743848,0.710553,0.489624,0.459506,0.153853,0.503926,0.668170,0.593723,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4657,NIH20F9153559,0.0,0.5,0.0,0.5,0.5,0.0,0.0,0.0,0.5,...,0.0,0.601687,0.610927,0.248576,0.331117,0.347169,0.346164,0.655244,0.591914,0
4658,NIH20F9086812,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.0,0.638227,0.550859,0.369780,0.390710,0.377312,0.394957,0.623123,0.649261,0
4659,NIH20F9155457,1.0,1.0,0.0,0.0,0.5,0.5,0.5,0.5,0.0,...,1.0,0.743848,0.699971,0.309813,0.351878,0.208765,0.584430,0.649345,0.626628,0
4660,NIH20F9670221,0.5,0.5,0.0,0.0,0.5,0.0,0.0,0.0,1.0,...,0.0,0.811163,0.761811,0.274432,0.361908,0.429825,0.236606,0.673387,0.857027,1
